# Analyzing borrowers’ risk of defaulting

By Yevgenii Atlasman  
October 2021

Our project is to prepare a report for a bank's loan division. We'll need to find out if a customer's marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers' credit worthiness.

**The main aim of the project is to build a credit scoring of a potential customer. The credit scoring is used to evaluate the ability of a potential borrower to repay their loan.**  

**Plan of our work**
 1. Preprocess the data:
 - Identify and fill in missing values
 - Replace the real number data type with the integer type
 - Delete duplicate data
 - Categorize the data
 2. Answer these questions:
 - Is there a connection between having kids and repaying a loan on time?
 - Is there a connection between marital status and repaying a loan on time?
 - Is there a connection between income level and repaying a loan on time?
 - How do different loan purposes affect on-time loan repayment?

## Open the data file and have a look at the general information.

Import libraries. Install `pymystem3` module.

Let's load the Mystem library from pymystem3 and call the Counter container from the collections module.

In [35]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

**Use try-except method to read csv file, because it should work both on local computer and jupiter hub.**   

In [38]:
try:
    credit_scoring = pd.read_csv('credit_scoring_eng.csv')
except:
    credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [39]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


It seems like there are a lot of negative values in the `days_employed` column. Probably it was mistake in inserting data. We should take a closer look at this column, and check how many values there negeative. Also in the column `purpose` we can see reasons for taking out a loan like `purchase of a car`, `buying my own car`, `to buy a car`, `car purchase`, so we'll need to unite all this values. Probably, customers were asked to write down purposes instead of making them choose from the list (with option write their own answer if they didn't find relevant purpose). 

## Data preprocessing

Let's check what columns have missing values and what data type in each column by calling the `info()` method.

In [40]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There are some missing values in columns `total_income` and `days_employed`. Take a closer look at these missing values later. There is also float type in `total_income`. It will be better to work with money as integer values.

Let's also take a look at first ten rows.

In [41]:
credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


There are some answers written in the higher case in `education` column. We'll need to make all the characters in the string lowercase later.

Let's also check how many rows in `days_employed` column have negative value.

In [42]:
credit_scoring[credit_scoring['days_employed'] < 0].count()

children            15906
days_employed       15906
dob_years           15906
education           15906
education_id        15906
family_status       15906
family_status_id    15906
gender              15906
income_type         15906
debt                15906
total_income        15906
purpose             15906
dtype: int64

`15906` negative vales from total `21525`. That's a lot. We should notify data engineers about this problem in order to find the root of it. We won't need this column in our analysis, so we can leave it as it is. 

Let's see if there are some other columns with negative values.

In [43]:
credit_scoring[credit_scoring['children'] < 0].count()

children            47
days_employed       44
dob_years           47
education           47
education_id        47
family_status       47
family_status_id    47
gender              47
income_type         47
debt                47
total_income        44
purpose             47
dtype: int64

There are 47 rows that have negative value in the column `children`.

Let's look what variables has column `children` using the `unique()` method.

In [44]:
credit_scoring['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

There are 47 customers with `-1` children. It's probably a mistake. Also we found out that there are customers with `20` children. We should change it to `1` and `2`, it might be a mistake in data entry.

**Use the `abs()` method to change all values in `children` to positive values.**   

In [45]:
credit_scoring['children'] = abs(credit_scoring['children'])
credit_scoring.loc[credit_scoring['children'] == 20, 'children'] = 2
credit_scoring['children'].unique()

array([1, 0, 3, 2, 4, 5])

As we saw previously, there were some values in the `education` column with differrent cases. Let's check how many different unique values in this column.

In [46]:
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

These are the same answers, only written with higher case letters. 

Let's make every variables in `education` column lowercase using the `str.lower()` method. And check the result using the `unique()` method.

In [47]:
credit_scoring['education'] = credit_scoring['education'].str.lower()

In [48]:
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Let's check if where are other columns which have `0` values creating the loop.

In [49]:
for i in credit_scoring:
    print(i, len(credit_scoring[credit_scoring[i]==0]))

children 14149
days_employed 0
dob_years 101
education 0
education_id 5260
family_status 0
family_status_id 12380
gender 0
income_type 0
debt 19784
total_income 0
purpose 0


It's fine with `children`, `education_id`, `family_status_id` and `debt` to have `0`. But for customers ages in column `dob_years` it's not. But we can skip this column, we are not going to use it in our analysis.

### Processing missing values

Let's see how many rows we have and how many missing values are in each column.

In [50]:
print('Total rows', len(credit_scoring))
credit_scoring.isnull().sum()

Total rows 21525


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

We can assume that people who didn't indicate their income in `total_income`, don't have income and these are the same people whose `days_employed` is null (they are unemployed). These are quantitative variables.

**We should check percentage of missing values in every column. For that count all the missing values in columns and divide amount of missing values by total number of rows and multiply by 100.**

In [51]:
report = credit_scoring.isna().sum().to_frame() 
report = report.rename(columns = {0: 'missing_values'}) 
report['% of total'] = ((report['missing_values'] / credit_scoring.shape[0])*100).round(2)
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
days_employed,2174,10.1
total_income,2174,10.1
children,0,0.0
dob_years,0,0.0
education,0,0.0
education_id,0,0.0
family_status,0,0.0
family_status_id,0,0.0
gender,0,0.0
income_type,0,0.0


**It's 10% of missing values in each column. We can try and replace it by some median values. But if it was more than 10%, we couldn't do it.**

Let's see if it's true. Use the `isnull()` method for finding missing values and the `unique()` method for seeing all unique values in a column.

In [52]:
credit_scoring[credit_scoring['total_income'].isnull()]['days_employed'].unique()

array([nan])

That's correct.

Let's look at these rows.

In [53]:
credit_scoring[credit_scoring['days_employed'].isnull()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


It's strange, but people with no information in `days_employed` and `total_income` still have `income_type` mentioned.

Let's see which different income types is in our dataset.

In [54]:
credit_scoring['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Let's take a look at unemployed customers.

In [55]:
credit_scoring.loc[credit_scoring['income_type'] == 'unemployed']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
14798,0,395302.838654,45,bachelor's degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


They are unemloyed, but have income (maybe passive income or some allowance).

#### Conclusion
We don't know for sure how the data was collected and what `unemployed` really means. Some people that are technically unemployed can still have income. We can skip values of `days_employed` column, because we are not going to use it. We should notify data engineers about problem in this column in order to find the root of it. We won't drop this column, because we still need to look for duplicates rows and it can change the result.

### Filling in missing values
Let's try to fill the `total_income` column by averages for some categories. Now, in order to fill in missing values in `total_income` column, lets find median total income value grouping by `income_type` column. Use the `median()` and the `groupby()` methods.

In [56]:
grouped1 = credit_scoring.groupby(['income_type'])['total_income'].median()
grouped1

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

Here we see median income for each `income_type`. Now we need to fill in all missing values in `total_income `according to values for `income_type` above. Use the `fillna()` method and `transform()`.

In [57]:
credit_scoring['clean_income1'] = credit_scoring['total_income'].fillna(credit_scoring.groupby([
    'income_type'])['total_income'].transform('median'))

In [58]:
credit_scoring[credit_scoring.total_income.isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,18962.3180
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,24071.6695
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,18962.3180
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,24071.6695
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,18962.3180


Now, let's do the same order finding out median values grouping by a. `income_type` and `education_id`. b. `income_type`, `education_id` and `gender`, because men's and women's salary could be different for the same education and income type. We should check it. 

In [59]:
grouped2 = credit_scoring.groupby(['income_type', 'education_id'])['total_income'].median()
grouped2

income_type                  education_id
business                     0               32285.6640
                             1               25451.3100
                             2               28778.7440
                             3               21887.8250
civil servant                0               27601.7775
                             1               21864.4750
                             2               25694.7750
                             3               23734.2870
                             4               17822.7570
employee                     0               26502.5190
                             1               21848.8175
                             2               24209.4300
                             3               20159.1860
                             4               31771.3210
entrepreneur                 0               79866.1030
paternity / maternity leave  1                8612.6610
retiree                      0               23078.5230
      

How we can see above civil servants with graduate degree have the smallest salary in their category, but employees with graduate degree have the biggest one in employee category and also graduate degree wins in retiree category.

In [60]:
credit_scoring['clean_income2'] = credit_scoring['total_income'].fillna(credit_scoring.groupby([
    'income_type', 'education_id'])['total_income'].transform('median'))

Check the result.

In [62]:
credit_scoring[credit_scoring.total_income.isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,18962.3180,18374.857
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,24071.6695,21864.475
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,18962.3180,18374.857
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,24071.6695,21864.475
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,18962.3180,18374.857


Group by `income_type`, `education_id` and `gender`.

In [63]:
grouped3 = credit_scoring.groupby(['income_type', 'gender', 'education_id'])['total_income'].median()
grouped3

income_type                  gender  education_id
business                     F       0               30382.9540
                                     1               23181.0850
                                     2               27275.4595
                                     3               21441.9210
                             M       0               37290.3815
                                     1               29102.2430
                                     2               30606.7090
                                     3               24016.1540
                             XNA     2               32624.8250
civil servant                F       0               25254.6300
                                     1               20133.0890
                                     2               21855.8610
                                     3               14339.0340
                                     4               17822.7570
                             M       0               3

Here we see that men with the same education and income type get much bigger salary than women. Only in category retiree-graduate degree women have bigger salary (40868) than men (15800) and entrepreneur women have (79866) and there isn't men here. But it's only outliers because of small sample of values in this group.

Check this for entrepreneurs.

In [64]:
credit_scoring[credit_scoring['income_type'] == 'entrepreneur']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,79866.103,79866.103
18697,0,-520.848083,27,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding,79866.103,79866.103


Yes, it's only two loaners with income type as entrepreneur. So we can't compare it.

In [65]:
credit_scoring['clean_income3'] = credit_scoring['total_income'].fillna(credit_scoring.groupby([
    'income_type', 'gender', 'education_id'])['total_income'].transform('median'))

Check the result.

In [66]:
credit_scoring[credit_scoring.total_income.isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,18962.3180,18374.857,20175.551
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,24071.6695,21864.475,27144.313
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,18962.3180,18374.857,18046.556
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,24071.6695,21864.475,20133.089
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,18962.3180,18374.857,18046.556


Let's check now median values for each `clean_income1`, `clean_income2`, `clean_income3`, and if we got all our missing values filled.

In [67]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,clean_income1,clean_income2,clean_income3
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000,21525.000000,21525.000000,21524.000000
mean,0.479721,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355,26436.051922,26456.597412,26456.873757
std,0.755528,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632,15686.986477,15703.769010,15723.367222
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000,3306.762000,3306.762000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500,17247.708000,17235.090000,17234.415000
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000,22815.103500,22959.405000,23181.085000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000,31287.991000,31703.887000,31456.873250
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000,362496.645000,362496.645000,362496.645000


#### Conclusion

**We decided to fill in missing values because the percentage of missing values 10% from all rows of data. We think that it could be some mistakes in inserting the data. Because it was the same amount of missing values in `days_employed` column and `total_income`, it might be questionnaire form on the web site or on the bank clerk computer, where then you leave `days_employed` column empty, you can't get the question about `total_income` we should check it with data engineers. But for our goal it's OK for now to fill in this values.**   

As we can see for clean_income3 where we looked for median value by `income_type`, `education_id` and `gender`, we've got closer value (23181) to median in `total_income` (23202). But we left one missing value apparantly in the entrepreneur-male category. It has small sample for men and women. So let's stick with filling in according only to `income_type` and `education_id`. The results obtained are quite close to original data.

Common logic is that it is better to think about the strategies of filling in the missing values and if it is impossible to fill in the missing values we can simply leave missing values in the data unchanged, because sometimes when the percentage of the missing values is big and the data is rather sensitive our attempts to fill in missing values using mean or median can be risky in terms of distortion of the original data, so it is important to take it into consideration!
       
We can use median to fill in missing values in column `total_income` because this statistics is more resistant to outliers. 

Finally is important to understand the nature of the missing values, formulate hypotheses or reasons for the presence of missing values in the data, and based on these hypotheses, try to choose a way to fill the data. It is important to understand that replacing missing values with a simple median or mean by column can significantly distort the original data if the percentage of gaps in the columns exceeds 10%.

### Data type replacement

Now, when we filled in missing values we can change data type in column `clean_income2` from float to integer. It's easier way to work with numbers like salary. We can use the `astype()` method with it's argument `('int')`.

In [68]:
credit_scoring['clean_income2'] = credit_scoring['clean_income2'].astype('int')

In [69]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40620.102,40620,40620.102
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,17932.802,17932,17932.802
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,23341.752,23341,23341.752
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,42820.568,42820,42820.568
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,25378.572,25378,25378.572


**Let's change data type of `days_employed` column too. For that we should fill in missing values with `0` first. Use the `fillna()` method for this column.**

In [70]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(0)

**Use the `astype()` for `days_employed` column with argument `int`.**

In [71]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].astype('int')

**Get rid from negative values in this column.**

In [72]:
credit_scoring['days_employed'] = abs(credit_scoring['days_employed'])

**Let's check how it looks like now.**

In [73]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40620.102,40620,40620.102
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,17932.802,17932,17932.802
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,23341.752,23341,23341.752
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,42820.568,42820,42820.568
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,25378.572,25378,25378.572


#### Conclusion

Now it's better to work with salary when it's type integer. And less mess with `days_employed`.

In [74]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
 12  clean_income1     21525 non-null  float64
 13  clean_income2     21525 non-null  int64  
 14  clean_income3     21524 non-null  float64
dtypes: float64(3), int64(7), object(5)
memory usage: 2.5+ MB


### Processing duplicates

Let's check if we have duplicates in our dataset using the `duplicated()` method for finding duplicates along with the method `sum()` that returns the total number of duplicates.

In [75]:
credit_scoring.duplicated().sum()

71

Check the result. Cut all the duplicates and put it in the table sorting by the `dob_years`.

In [76]:
credit_scoring.loc[credit_scoring.duplicated(keep=False), :].sort_values('dob_years').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3
20297,1,0,23,secondary education,1,civil partnership,1,F,employee,0,NaN,to have a wedding,22815.1035,21848,19860.334
8853,1,0,23,secondary education,1,civil partnership,1,F,employee,0,NaN,to have a wedding,22815.1035,21848,19860.334
15892,0,0,23,secondary education,1,unmarried,4,F,employee,0,NaN,second-hand car purchase,22815.1035,21848,19860.334
19321,0,0,23,secondary education,1,unmarried,4,F,employee,0,NaN,second-hand car purchase,22815.1035,21848,19860.334
3452,0,0,29,bachelor's degree,0,married,0,M,employee,0,NaN,buy residential real estate,22815.1035,26502,30759.568


Let's drop duplicates using the `drop_duplicates()` method and the `reset_index()`.

In [77]:
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True)
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40620.102,40620,40620.102
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,17932.802,17932,17932.802
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,23341.752,23341,23341.752
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,42820.568,42820,42820.568
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,25378.572,25378,25378.572


#### Conclusion

As we can see there are 3 duplicate rows for the customer in age 71, who has secondary education, in civil partnership, female, retired, and needs loan fot having a wedding. We can't be sure that there was triple entry from the same customer and not from different 71-years old women. We should check how data was collected with different departments, ask data engineers. In this case we can drop these duplicates. There are only 71, so it won't affect on the total result.

### Categorizing Data

We need to make categorization - to organize selected data into groups according to specific criterias. We'll create a list of categories from the given data.

Let's categorize our customers according to their's income. Call the `describe()` method to `total_income` column.

In [78]:
credit_scoring.clean_income2.describe()

count     21454.000000
mean      26472.049128
std       15726.191978
min        3306.000000
25%       17202.000000
50%       22993.000000
75%       31728.750000
max      362496.000000
Name: clean_income2, dtype: float64

Write function using the keyword `def`. Let's accept that income from 0 to 10000 equal to `poverty` category, from 10000 to 23000 - `poor`, from 23000 to 50000 - `middle-class`, from 50000 to 100000 - `wealthy`, from 100000 - `rich`.

In [79]:
def clean_income2_cat(x):
    if x>=0 and x<=10000:
        return 'poverty'
    if x>10000 and x<=23000:
        return 'poor'
    if x>23000 and x<=50000:
        return 'middle-class'
    if x>50000 and x<=100000:
        return 'wealthy'
    if x>100000:
        return 'rich'

Check how our function works by creating the new column and applying the function to column `clean_income2`. Use the `apply()` method. And print dataframe.

In [80]:
credit_scoring['income_level'] = credit_scoring['clean_income2'].apply(clean_income2_cat)

In [81]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3,income_level
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40620.102,40620,40620.102,middle-class
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,17932.802,17932,17932.802,poor
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,23341.752,23341,23341.752,middle-class
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,42820.568,42820,42820.568,middle-class
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,25378.572,25378,25378.572,middle-class


In [82]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  int64  
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
 12  clean_income1     21454 non-null  float64
 13  clean_income2     21454 non-null  int64  
 14  clean_income3     21453 non-null  float64
 15  income_level      21454 non-null  object 
dtypes: float64(3), int64(7), object(6)
memor

Check how many customers in the each category grouping by `income_level` and using the `count()` method.

In [83]:
credit_scoring.groupby('income_level')['income_level'].count()

income_level
middle-class    9403
poor            9803
poverty          927
rich              99
wealthy         1222
Name: income_level, dtype: int64

Let's categorize our dataset according to purposes. We should check what purposes we have in the `purpose` column by calling the `unique()` method for this column.

In [84]:
credit_scoring.purpose.unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Now, we can see all the purposes, so we can create 4 categories and make list of key words for each of them.

In [85]:
wedding_category = ['wedding','ceremony']
house_category = ['house', 'real','estate','property','housing']
car_category = ['car','cars']
education_category = ['education','university', 'educated']

In [86]:
m = Mystem()

Let's build up the function for lemmatization words and check if it works properly.

In [87]:
def lemmatization_func(line):
    lemmatized = m.lemmatize(line)
    return lemmatized

Let's check how it works. Take example of sentence from the column `purpose` from the first row and save into variable `example`.

In [88]:
example = credit_scoring.loc[0]['purpose']
example

'purchase of the house'

Let's lemmatize this example.

In [89]:
lemmatization_func(example)

['purchase', ' ', 'of', ' ', 'the', ' ', 'house', '\n']

Now write and check `any()` function. It should return `True`, if it finds any word in `lemmatization_func(example)` that matches any word from list of words in variable `house_category`.

In [90]:
any(word in lemmatization_func(example) for word in house_category)

True

Here write the same for list of words in variable `wedding_category`. It should return `False`, because in our variable `example` it won't find any word that matches any word from list of words in `wedding_category`.

In [91]:
any(word in lemmatization_func(example) for word in wedding_category)

False

It works properly.

Now we should modify our `lemmatization_func(line)` function, so that it will check lemmatized words in each category list and will return name of category like `wedding`, `car`, `education`, `house` or `other` if it's doesn't match any words from categories lists.

In [92]:
def lemmatization_func(line):
    lemmatized = m.lemmatize(line)
    if any(word in lemmatized for word in wedding_category):
        return 'wedding'
    elif any(word in lemmatized for word in car_category):
        return 'car'
    elif any(word in lemmatized for word in education_category):
        return 'education'
    elif any(word in lemmatized for word in house_category):
        return 'house'
    else:
        return 'other'

Let's add new column `clean_purpose` to our dataset in order more clearly categorize loan puproses.

In [93]:
credit_scoring['clean_purpose'] = credit_scoring['purpose'].apply(lemmatization_func)

Now, check how many loans were for different purpose categories.

In [95]:
credit_scoring['clean_purpose'].value_counts()

house        10811
car           4306
education     4013
wedding       2324
Name: clean_purpose, dtype: int64

We categorized all our purposes into just 4 categories. Let's check our dataset.

In [98]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,clean_income1,clean_income2,clean_income3,income_level,clean_purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40620.102,40620,40620.102,middle-class,house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,17932.802,17932,17932.802,poor,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,23341.752,23341,23341.752,middle-class,house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,42820.568,42820,42820.568,middle-class,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,25378.572,25378,25378.572,middle-class,wedding


#### Conclusion

We categorized our customers by income and purpose to get loan. Now we can group them to different categories by their key characteristic. According to this categories we will be able to analyse relation between different categories of customers and their ability to repay a loan on time.

## Answer the questions

### Is there a relation between having kids and repaying a loan on time?

Build pivot table using `pivot_table(data, index, columns, values, aggfunc, margins=True)` method, where `index`: the column or columns whose values become the indices in the pivot table, `columns`: the column or columns whose values become columns in the pivot table, `values`: the values we want to aggregate in the pivot table, `aggfunc`: the aggregate function that we're applying to the values, `margins`: the argument to include row/column totals.

Let's look how having kids influnce on repaying a loan on time. Where `0` in debt means that loan was repayed on time, and `1` in debt means that loan was defaulted.

In [99]:
children_status = pd.pivot_table(credit_scoring, index = 'children', columns = 'debt', 
                              values = 'family_status_id', aggfunc = 'count', margins=True)
children_status

debt,0,1,All
children,,,
0,13028.0,1063.0,14091
1,4410.0,445.0,4855
2,1926.0,202.0,2128
3,303.0,27.0,330
4,37.0,4.0,41
5,9.0,NaN,9
All,19713.0,1741.0,21454


Calculate `default_rate` for customers with different children status and apply new column to our pivot table (`default_rate`). For this divide column with defaulted loans (has index `[1]`, because column `children` became index in our pivot table) by overall amount of loans for each amount of kids (`[All]`).

In [100]:
children_status['default_rate'] = (children_status[1] / children_status['All']) * 100
children_status

debt,0,1,All,default_rate
children,,,,
0,13028.0,1063.0,14091,7.543822
1,4410.0,445.0,4855,9.165808
2,1926.0,202.0,2128,9.492481
3,303.0,27.0,330,8.181818
4,37.0,4.0,41,9.756098
5,9.0,NaN,9,NaN
All,19713.0,1741.0,21454,8.115037


#### Conclusion

People with ***no children*** are the **best category** in term to the `default rate`, because their `default rate - 7.54%` below the `average default rate - 8.12%`. The **worst category** - people with ***4 children*** (`9.76%`). And with increasing the amount of children the default rate also increasing. It means that ***more children*** - **higher default risk**. Only people with 3 children are out of this rule and people with 5 children had no defaults. But we should keep in mind that it was only 9 people with 5 kids who took a loan. So it can be too small sample. Although maybe people with 5 children more responsible, because they know that have a lot of children to care about.

**Recomendation** to the bank will be to check better people with more children. Our conclusion that people without children the most trusted group.

### Is there a relation between marital status and repaying a loan on time?

Build pivot table to see how different **marital status** influnces on repaying a loan on time. 

In [101]:
family_status = pd.pivot_table(credit_scoring, index = 'family_status', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins=True)
family_status

debt,0,1,All
family_status,,,
civil partnership,3763,388,4151
divorced,1110,85,1195
married,11408,931,12339
unmarried,2536,274,2810
widow / widower,896,63,959
All,19713,1741,21454


Calculate `default_rate` for customers with different marital status.

In [102]:
family_status['default_rate'] = (family_status[1] / family_status['All']) * 100
family_status

debt,0,1,All,default_rate
family_status,,,,
civil partnership,3763,388,4151,9.347145
divorced,1110,85,1195,7.112971
married,11408,931,12339,7.545182
unmarried,2536,274,2810,9.750890
widow / widower,896,63,959,6.569343
All,19713,1741,21454,8.115037


Let's double check our result by grouping by `children` and `family_status` in pivot table.

In [104]:
family_status_kids = pd.pivot_table(credit_scoring, index = ['family_status','children'], columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins=True)
family_status_kids

debt                              0       1    All
family_status     children                        
civil partnership 0          2501.0   229.0   2730
                  1           882.0   118.0   1000
                  2           322.0    33.0    355
                  3            48.0     8.0     56
                  4             8.0     NaN      8
                  5             2.0     NaN      2
divorced          0           729.0    55.0    784
                  1           295.0    21.0    316
                  2            75.0     8.0     83
                  3            10.0     1.0     11
                  4             1.0     NaN      1
married           0          6952.0   516.0   7468
                  1          2757.0   247.0   3004
                  2          1434.0   148.0   1582
                  3           232.0    17.0    249
                  4            26.0     3.0     29
                  5             7.0     NaN      7
unmarried         0          2052.0   210.0   2262
                  1           402.0    52.0    454
                  2            74.0    10.0     84
                  3             7.0     1.0      8
                  4             1.0     1.0      2
widow / widower   0           794.0    53.0    847
                  1            74.0     7.0     81
                  2            21.0     3.0     24
                  3             6.0     NaN      6
                  4             1.0     NaN      1
All                         19713.0  1741.0  21454

In [105]:
family_status_kids['default_rate'] = (family_status_kids[1] / family_status_kids['All']) * 100
family_status_kids

debt                              0       1    All  default_rate
family_status     children                                      
civil partnership 0          2501.0   229.0   2730      8.388278
                  1           882.0   118.0   1000     11.800000
                  2           322.0    33.0    355      9.295775
                  3            48.0     8.0     56     14.285714
                  4             8.0     NaN      8           NaN
                  5             2.0     NaN      2           NaN
divorced          0           729.0    55.0    784      7.015306
                  1           295.0    21.0    316      6.645570
                  2            75.0     8.0     83      9.638554
                  3            10.0     1.0     11      9.090909
                  4             1.0     NaN      1           NaN
married           0          6952.0   516.0   7468      6.909480
                  1          2757.0   247.0   3004      8.222370
                  2          1434.0   148.0   1582      9.355247
                  3           232.0    17.0    249      6.827309
                  4            26.0     3.0     29     10.344828
                  5             7.0     NaN      7           NaN
unmarried         0          2052.0   210.0   2262      9.283820
                  1           402.0    52.0    454     11.453744
                  2            74.0    10.0     84     11.904762
                  3             7.0     1.0      8     12.500000
                  4             1.0     1.0      2     50.000000
widow / widower   0           794.0    53.0    847      6.257379
                  1            74.0     7.0     81      8.641975
                  2            21.0     3.0     24     12.500000
                  3             6.0     NaN      6           NaN
                  4             1.0     NaN      1           NaN
All                         19713.0  1741.0  21454      8.115037

#### Conclusion

**Married, divorced people and widow/widower** the most trusted categories, because of lower default rate (below the `average 8.12%`). People in ***civil partnership*** and ***unmarried*** have **high default rate** and have to be checked carefully. We see that unmarried people have high default rate especially with 4 kids - 50%, widows/widowers with 2 children also have high default rate -12.5%. People in civil relationships also have high default risk. Probably the **best option** for bank it's ***married people with no kids or 1 kid***. And also divorced without kids or with 1 kid and widow/widower without kids.

### Is there a relation between income level and repaying a loan on time?

Build pivot table for different income level to find out how it influences on repaying a loan.

In [107]:
income_level = pd.pivot_table(credit_scoring, index = 'income_level', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins=True)
income_level

debt,0,1,All
income_level,,,
middle-class,8666,737,9403
poor,8949,854,9803
poverty,869,58,927
rich,93,6,99
wealthy,1136,86,1222
All,19713,1741,21454


Calculate default rate for people with different income level.

In [108]:
income_level['default_rate'] = (income_level[1] / income_level['All']) * 100
income_level

debt,0,1,All,default_rate
income_level,,,,
middle-class,8666,737,9403,7.837924
poor,8949,854,9803,8.711619
poverty,869,58,927,6.256742
rich,93,6,99,6.060606
wealthy,1136,86,1222,7.037643
All,19713,1741,21454,8.115037


#### Conclusion

People with income level group like **rich, wealthy and poverty** are repaying their loans better and have **lower default rate** than poor and middle-class. Maybe people in `poverty` category take less money for loan or repay carefully and safe money to return it to the bank. Maybe people with small total income have less kids. We don't know if it's true overall, but for our dataset we've got this result.

### How do different loan purposes affect on-time repayment of the loan?

Build pivot table for different loan purposes to find out how it influences on repaying a loan.

In [109]:
loan_purpose = pd.pivot_table(credit_scoring, index = 'clean_purpose', columns = 'debt', 
                               values = 'family_status_id', aggfunc = 'count', margins=True)
loan_purpose

debt,0,1,All
clean_purpose,,,
car,3903,403,4306
education,3643,370,4013
house,10029,782,10811
wedding,2138,186,2324
All,19713,1741,21454


Calculate `default rate` for people with different loan purposes.

In [110]:
loan_purpose['default_rate'] = (loan_purpose[1] / loan_purpose['All']) * 100
loan_purpose

debt,0,1,All,default_rate
clean_purpose,,,,
car,3903,403,4306,9.359034
education,3643,370,4013,9.220035
house,10029,782,10811,7.233373
wedding,2138,186,2324,8.003442
All,19713,1741,21454,8.115037


#### Conclusion

**Lower default risk** have people who take ***loans for houses***. And such purposes like ***buying car and education*** have **high risk** not repaying loans on time. Probably, students have less income level, have less time to work or maybe don't have degree yet to work on high-payed job. And people who buy real estate can rent it and get passive income, because a lot of people who buy property already have some money and want to invest it.

## General conclusion

We examined and prepared our data to analysis. We found out in data some strange values like negative `days_employed` column and `0` values in the customer's age. Probably it was mistake in inserting data. So we would notify engineers about that and skiped this columns, because we shouldn't use these. Also we had weird numbers of children like `-1` and `20`. We thougt that it's typos and replaced it with `1` and `2`.

We found missing values in `total_income`. We grouped all customers by `income_type` and `education` (we've got closer result to original data than other combination). We found median total income for each group. So we could fill missing values in with median values.

After that we transofmed all education values to lowercase because it was too messed up. We replaced data type of salary to integer so it would be cleaner and easier to work with it. We found out 71 duplicate rows, so we delated it.

We categorized customers by their's total income to 5 groupes: `poverty` with income `less than 10000`, `poor` with income `from 10000 to 23000`, `middle-class` with income `from 23000 to 50000`, `wealthy` with income `from 50000 to 100000` and `rich` - `more than 100000`. And got `9803 loaners in poor class`, `9403 in middle-class`, `927 in poverty class`, `1222 - wealthy` and `99 - rich`.

We categorized customers by their loan purpose on `4 groups` - `car`, `house`, `wedding` and `education`. And we've got `10811` loaners who took a loan `to buy house`, `4306 - to buy car`, `4013 - on education` and `2324 - on wedding`.

Answering all of 4 questions we figured out that the **best categories** in terms of the loan are ***married people (default rate - 7.55%), without children (default rate - 7.55% too), whose total income more than 50000 (default rate - from 6 to 7%)*** and ***middle class with total income more than 23000 (default rate - 7.83%), who want to buy house (default rate - 7.23%)***. These are the most `truthworthy` loaners.